In [54]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
plt.rcParams['animation.ffmpeg_path'] = r"C:\Users\rfgla\Documents\Ray\ffmpeg-master-latest-win64-gpl\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"
import torch
from torchvision.io import read_video, read_video_timestamps
import sys
sys.path.append(r'C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\VideoGPT-master')
from videogpt import download, VQVAE, VideoGPT
from videogpt.data import preprocess

In [55]:
# Load in video gpt model
gpt_path = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\VideoGPT-master\lightning_logs\videoGPT_gesture_sorted_data_resized\checkpoints\epoch=5-step=18539.ckpt"
device = torch.device('cuda')
gpt = VideoGPT.load_from_checkpoint(gpt_path).to(device)

In [56]:
# Load in video data
video_filename = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\data\gesture_sorted_data_resized\test\EFL\103_18_0_4_1_stand.mp4"
resolution, sequence_length = gpt.args.resolution, 16
pts = read_video_timestamps(video_filename, pts_unit='sec')[0]
video = read_video(video_filename, pts_unit='sec', start_pts=pts[0], end_pts=pts[sequence_length - 1])[0]
video = preprocess(video, resolution, sequence_length).unsqueeze(0).to(device)
print(torch.max(video), torch.min(video))
print(video.shape)

tensor(0.4989, device='cuda:0') tensor(-0.5000, device='cuda:0')
torch.Size([1, 3, 16, 64, 64])


# Visualize the input video
# Visualize the reconstruction
print(video.shape)
disp_video = video.copy()
disp_video = disp_video[0].permute(1, 2, 3, 0)  # CTHW -> THWC
videos = ((disp_video + 0.5) * 255).cpu().numpy().astype('uint8')
print(videos.shape)

fig = plt.figure()
plt.axis('off')
im = plt.imshow(videos[0, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :])

def animate(i):
    im.set_data(videos[i, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())

In [57]:
# Get the label for the data
if video_filename.find("gesture_sorted_data") != -1:  # Folders will be for each separate gesture
    classes = ["EFL", "EFR", "SFL", "SFR", "SAL", "SAR", "SFE", "STL", "STR"]
else:  # Folders will be for both left and right variants of each gesture
    classes = ["EF", "SF", "SA", "SFE", "ST"]

for l, this_label in enumerate(classes):
    if video_filename.find(this_label) != -1:
        label = torch.tensor([l])
        continue
print(label)

tensor([0])


In [61]:
label = torch.tensor([7])
n_iterations = 3  # Video will be n_iterations * sequence_length long
n_cond_frames = gpt.args.n_cond_frames
samples = [[] for _ in range(n_iterations)]
for this_iteration in range(n_iterations):
    if this_iteration != 0:
        video[:, :, :, :, :] = torch.flip(samples[this_iteration - 1], dims=[2])  # Update video to be last predicted frames played in reverse
    batch = {'video': video, 'label': label}
    samples[this_iteration] = gpt.sample(1, batch)

torch.Size([1, 3, 1, 64, 64])


  0%|                                                                                                                                                                                                         | 0/1024 [00:00<?, ?it/s]C:\Users\rfgla\anaconda3\envs\bme1570_env\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\rfgla\anaconda3\envs\bme1570_env\lib\site-packages\torch\utils\checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████

torch.Size([1, 3, 1, 64, 64])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:29<00:00, 35.06it/s]


torch.Size([1, 3, 1, 64, 64])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:19<00:00, 51.56it/s]


In [62]:
all_samples = torch.cat([this_sample for this_sample in samples], axis=2)
print(all_samples.shape)

torch.Size([1, 3, 48, 64, 64])


In [63]:
import math
import numpy as np

b, c, t, h, w = all_samples.shape
disp_samples = all_samples
print(disp_samples.shape)
disp_samples = disp_samples.permute(0, 2, 3, 4, 1)
disp_samples = (disp_samples.cpu().numpy() * 255).astype('uint8')

videos = disp_samples[0]

fig = plt.figure()
plt.title('ucf101 unconditional samples')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())

torch.Size([1, 3, 48, 64, 64])
